In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bbref_scraper import SCRAPER

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

2023-04-05 18:17:38.915596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Compare models by training prior to 2022 and testing on 2021-2022 season

In [68]:
scraper = SCRAPER(np.arange(2000, 2023))
data = scraper.load()
data['home'] = len(data) * [1.0]
train = data[data['year'] < 2022]
train = train[(train['G_home'] >= 25) & (train['G_home'] >= 25) & (train['Rest_home'] < 20) & (train['Rest_away'] < 20)]
test  = data[data['year'] == 2022]
test = test[(test['G_home'] > 5) & (test['G_home'] > 5)]
X_train, y_train = np.array(train[scraper.features]), np.array(train['W_home'])
X_test, y_test   = np.array(test[scraper.features]),  np.array(test['W_home'])

In [49]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_shape = (X_train.shape[1],)))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1,  activation = 'sigmoid'))

In [50]:
# Compile the model
model.compile(loss      = 'binary_crossentropy',
              optimizer = 'adam',
              metrics   = ['accuracy']
              )

In [51]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 25, batch_size = 32)

Epoch 1/25
518/518 [==============================] - 1s 763us/step - loss: 0.6579 - accuracy: 0.6483 - val_loss: 0.6591 - val_accuracy: 0.6205
Epoch 2/25
518/518 [==============================] - 0s 602us/step - loss: 0.6071 - accuracy: 0.6688 - val_loss: 0.6404 - val_accuracy: 0.6363
Epoch 3/25
518/518 [==============================] - 0s 598us/step - loss: 0.6020 - accuracy: 0.6736 - val_loss: 0.6508 - val_accuracy: 0.6314
Epoch 4/25
518/518 [==============================] - 0s 591us/step - loss: 0.5987 - accuracy: 0.6764 - val_loss: 0.6409 - val_accuracy: 0.6374
Epoch 5/25
518/518 [==============================] - 0s 589us/step - loss: 0.5970 - accuracy: 0.6783 - val_loss: 0.6413 - val_accuracy: 0.6422
Epoch 6/25
518/518 [==============================] - 0s 639us/step - loss: 0.5952 - accuracy: 0.6803 - val_loss: 0.6460 - val_accuracy: 0.6281
Epoch 7/25
518/518 [==============================] - 0s 636us/step - loss: 0.5943 - accuracy: 0.6818 - val_loss: 0.6414 - val_accuracy:

In [52]:
y_pred = model.predict(X_test)
accuracy_score(y_test, np.round(y_pred))

37/37 [==============================] - 0s 398us/step


0.6247833622183708

New model

In [73]:
X_train, y_train = np.array(train[['W/L%_home', 'W/L%_away', 'NRtg_home', 'NRtg_away']]), np.array(train['W_home'])
X_test, y_test   = np.array(test[['W/L%_home', 'W/L%_away', 'NRtg_home', 'NRtg_away']]),  np.array(test['W_home'])

X_train, y_train = np.array(train[['home']]), np.array(train['W_home'])
X_test, y_test   = np.array(test[['home']]),  np.array(test['W_home'])

In [74]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_shape = (X_train.shape[1],)))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1,  activation = 'sigmoid'))

# Compile the model
model.compile(loss      = 'binary_crossentropy',
              optimizer = 'adam',
              metrics   = ['accuracy']
              )

In [75]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 25, batch_size = 32)

Epoch 1/25
518/518 [==============================] - 1s 771us/step - loss: 0.6748 - accuracy: 0.5940 - val_loss: 0.6880 - val_accuracy: 0.5559
Epoch 2/25
518/518 [==============================] - 0s 592us/step - loss: 0.6736 - accuracy: 0.5993 - val_loss: 0.6918 - val_accuracy: 0.5559
Epoch 3/25
518/518 [==============================] - 0s 606us/step - loss: 0.6736 - accuracy: 0.5993 - val_loss: 0.6899 - val_accuracy: 0.5559
Epoch 4/25
518/518 [==============================] - 0s 600us/step - loss: 0.6735 - accuracy: 0.5993 - val_loss: 0.6910 - val_accuracy: 0.5559
Epoch 5/25
518/518 [==============================] - 0s 581us/step - loss: 0.6734 - accuracy: 0.5993 - val_loss: 0.6884 - val_accuracy: 0.5559
Epoch 6/25
518/518 [==============================] - 0s 578us/step - loss: 0.6735 - accuracy: 0.5993 - val_loss: 0.6943 - val_accuracy: 0.5559
Epoch 7/25
518/518 [==============================] - 0s 604us/step - loss: 0.6735 - accuracy: 0.5993 - val_loss: 0.6932 - val_accuracy:

In [62]:
test['records_diff'] = test['W/L%_home'] - test['W/L%_away']

In [65]:
test['pred'] = np.sign(test['records_diff'])
test['pred'] = test['pred'].replace(-1, 0)

In [67]:
accuracy_score(test['pred'], test['W_home'])

0.6195840554592721